In [71]:
import random
import numpy as np


class Board:
    def __init__(self, n):
        self.n_queen = n
        self.map = [[0 for j in range(n)] for i in range(n)]
        self.fit = 0

    def set_queens(self):
        for i in range(self.n_queen):
            j = random.randint(0, self.n_queen - 1)
            self.map[i][j] = 1

    def fitness(self):
        self.fit = self.n_queen * (self.n_queen - 1) // 2
        for i in range(self.n_queen):
            for j in range(self.n_queen):
                if self.map[i][j] == 1:
                    for k in range(1, self.n_queen - i):
                        if self.map[i + k][j] == 1:
                            self.fit -= 1
                        if j - k >= 0 and self.map[i + k][j - k] == 1:
                            self.fit -= 1
                        if j + k < self.n_queen and self.map[i + k][j + k] == 1:
                            self.fit -= 1

    def show(self):
        print(np.matrix(self.map))
        print("Fitness: ",  self.fit)

        
if __name__ == '__main__':
    test = Board(5)
    test.set_queens()
    test.fitness()
    test.show()

[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [0 0 0 1 0]]
Fitness:  6


In [105]:
class Hill_Climb:
    
    def __init__(self, n):
        self.step = 0
        self.map = n
        self.q_pos = set()  
    
    def moveQueen(self):
        temp = self.map.map[0]
        self.map.map[0] = [0,0,0,0,0]
        mfit = 0
        mindx = 0
        for i in range(5):
            self.map.map[i] = [0,0,0,0,0]
            mfit = 0
            mindx = 0
            for j in range(5):
                self.map.map[i][j] = 1
                self.map.fitness()
                if(self.map.fit > mfit):
                    mfit = self.map.fit
                    mindx = j
                self.map.map[i][j] = 0
                if(mfit == 10):
                    print("Finished")
                    self.map.fitness()
                    self.map.show()
                    return self
            self.map.map[i][mindx] = 1
        self.moveQueen()
        
if __name__ == '__main__':
    home = Board(5)
    home.set_queens()
    home.fitness()
    home.show()
    print("\n")
    climb = Hill_Climb(home)
    climb.moveQueen()
        
    

[[0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [1 0 0 0 0]]
Fitness:  5


Finished
[[0 0 1 0 0]
 [0 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 1 0]
 [1 0 0 0 0]]
Fitness:  10


In [103]:
class Genetic:
    def __init__(self, n):
        self.step = 0


if __name__ == '__main__':
    home = Board(5)
    home.show()

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
Fitness:  0


In [ ]:
 """
    #The original code makes it so that they cannot be on the same row 
    """
    def checkHorizontal(self,pos):
        conflicts = 0
        stop = False
        x = pos[0]
        y = pos[1]
        #Check left
        while(y > -1):
            y -= 1
            if(self.map[x][y] == 1):
                conflicts += 1
                break
        
        #Check right
        y = pos[1]
        while(y < 4):
                y += 1
                if(self.map[x][y] == 1):
                    conflicts += 1
                    break
        return conflicts
"""
        
    def checkDiagonal(self,pos):
        #Check left diagonal 
        conflicts = 0
        stop = False
        x = pos[0]
        y = pos[1]
        while(x > 0 and y > 0):
            x-= 1
            y-= 1
            if(self.map[x][y] == 1):
                conflicts += 1
                self.addAttacker(x,y,pos)
                break
        #Check right diagonal 
        stop = False
        x = pos[0]
        y = pos[1]
        while(x < 4 and y > 0):
            x+= 1
            y-= 1
            if(self.map[x][y] == 1):
                conflicts += 1
                self.addAttacker(x,y,pos)
                break
        
        #Check down left diagonal
        stop = False
        x = pos[0]
        y = pos[1]
        while(x > 0 and y < 4):
            x-= 1
            y+= 1
            if(self.map[x][y] == 1):
                conflicts += 1
                self.addAttacker(x,y,pos)
                break
        
        #Check down right diagonal 
        stop = False
        x = pos[0]
        y = pos[1]
        while(x < 4 and y < 4):
            x+= 1
            y+= 1
            if(self.map[x][y] == 1):
                conflicts += 1
                self.addAttacker(x,y,pos)
                break
        return conflicts
        
    def checkVertical(self,pos):
        #Check up
        conflicts = 0
        stop = False
        x = pos[0]
        y = pos[1]
        #Check left
        while(x > 0):
            x -= 1
            if(self.map[x][y] == 1):
                conflicts += 1
                self.addAttacker(x,y,pos)
                break
        
        #Check down 
        x = pos[0]
        while(x < 4):
                x += 1
                if(self.map[x][y] == 1):
                    conflicts += 1
                    self.addAttacker(x,y,pos)
                    break
        return conflicts
    
    def show(self):
        print(np.matrix(self.map))
        
    def addAttacker(self,x,y,pos):
        value =  str(x) + str(y)
        key = str(pos[0]) + str(pos[1])
        if(value in self.attackers and self.attackers[value] == key):
            return self
        self.attackers[key] = value
        ("print attacker was added")
        return self
""" 

In [60]:
for i in range(5):
    print(i)
    

0
1
2
3
4
